In [1]:
# !pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import ElementClickInterceptedException
import numpy as np
import streamlit as st
import pymysql

In [3]:
#10 states links
state_links=["https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/jksrtc"
]

# 1. Andhra - apsrtc

In [4]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [5]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        apsrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in apsrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [9]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada, Name: Hyderabad to Vijayawada
Link: https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad, Name: Vijayawada to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-ongole, Name: Hyderabad to Ongole
Link: https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam, Name: Kakinada to Visakhapatnam
Link: https://www.redbus.in/bus-tickets/bangalore-to-tirupathi, Name: Bangalore to Tirupati
Link: https://www.redbus.in/bus-tickets/bangalore-to-kadapa, Name: Bangalore to Kadapa
Link: https://www.redbus.in/bus-tickets/ongole-to-hyderabad, Name: Ongole to Hyderabad
Link: https://www.redbus.in/bus-tickets/kadapa-to-bangalore, Name: Kadapa to Bangalore
Link: https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore, Name: Chittoor (Andhra Pradesh) to Bangalore
Link: https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada, Name: Visakhapatnam to Kakinada
Link: https://www.redbus.in/bus-tickets/bangalore

In [10]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("andhra_route_details.csv")
df.shape

(46, 2)

In [11]:
def scrolling_a():
    scrolling = True
    while scrolling:
        old_page_source = driver_a.page_source
        driver_a.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_page_source = driver_a.page_source
        if new_page_source == old_page_source:
            scrolling = False

In [12]:
driver_a = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_a, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_a.get(route_link)
    time.sleep(2)
    
    route_links = driver_a.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)
        
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_a()

    bus_name = driver_a.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_a.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_a.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_a.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_a.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_a.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_a.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_a.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Successfully scraped using Selenium


In [13]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

4578 4578 4578 4578 4578 4578 4578 4578 4578 4578


In [14]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_andhra = pd.DataFrame(dict)       

In [15]:
df_andhra

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3634,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),10:10,05h 50m,16:00,4.0,INR 670,37 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3598,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:35,07h 00m,17:35,3.0,INR 436,32 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35086,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),11:15,05h 45m,17:00,3.9,INR 670,43 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3580,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),12:15,05h 45m,18:00,3.9,INR 670,43 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 24256,INDRA(A.C. Seater),12:30,05h 30m,18:00,3.0,INR 528,30 Seats available
...,...,...,...,...,...,...,...,...,...,...
4573,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,True Bus,Volvo Multi Axle B9R A/C Sleeper (2+1),17:00,06h 59m,23:59,3.2,INR 4400,36 Seats available
4574,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Muneer Travels,Non A/C Seater / Sleeper (2+1),22:15,05h 30m,03:45,3.0,INR 500,40 Seats available
4575,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,DHANUNJAYA TRAVELS,A/C Sleeper (2+1),21:00,06h 00m,03:00,2.2,INR 1000,36 Seats available
4576,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Meghana Travels,A/C Sleeper (2+1),22:20,05h 55m,04:15,1.8,2850,9 Seats available


In [16]:
df_andhra[df_andhra['Route_name']=='Hyderabad to Vijayawada']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3634,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),10:10,05h 50m,16:00,4.0,INR 670,37 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3598,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:35,07h 00m,17:35,3.0,INR 436,32 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35086,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),11:15,05h 45m,17:00,3.9,INR 670,43 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3580,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),12:15,05h 45m,18:00,3.9,INR 670,43 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 24256,INDRA(A.C. Seater),12:30,05h 30m,18:00,3.0,INR 528,30 Seats available
...,...,...,...,...,...,...,...,...,...,...
450,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Naveen Tours and Travels,A/C Seater/Sleeper (2+1),14:50,07h 25m,22:15,1.4,INR 3999,44 Seats available
451,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Naveen Tours and Travels,A/C Seater/Sleeper (2+1),20:50,07h 25m,04:15,1.4,INR 3999,44 Seats available
452,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Naveen Tours and Travels,A/C Seater/Sleeper (2+1),22:00,08h 15m,06:15,1.4,INR 3898,44 Seats available
453,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Shivanjaneya Travels,A/C Seater / Sleeper (2+1),22:00,06h 00m,04:00,,4750,40 Seats available


In [17]:
df_andhra.to_csv('Andhra.csv')

# 2. Kerala - Kerala RTC

In [78]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [79]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1,6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        krtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in krtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [84]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/bangalore-to-kozhikode, Name: Bangalore to Kozhikode
Link: https://www.redbus.in/bus-tickets/kozhikode-to-bangalore, Name: Kozhikode to Bangalore
Link: https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam, Name: Kozhikode to Ernakulam
Link: https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode, Name: Ernakulam to Kozhikode
Link: https://www.redbus.in/bus-tickets/bangalore-to-kannur, Name: Bangalore to Kannur
Link: https://www.redbus.in/bus-tickets/kozhikode-to-mysore, Name: Kozhikode to Mysore
Link: https://www.redbus.in/bus-tickets/kannur-to-bangalore, Name: Kannur to Bangalore
Link: https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram, Name: Kozhikode to Thiruvananthapuram
Link: https://www.redbus.in/bus-tickets/mysore-to-kozhikode, Name: Mysore to Kozhikode
Link: https://www.redbus.in/bus-tickets/bangalore-to-kalpetta, Name: Bangalore to Kalpetta (kerala)
Link: https://www.redbus.in/bus-tickets/kalpetta-to-bangalore, Name: 

In [85]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("kerala_route_details.csv")
df.shape

(15, 2)

In [86]:
def scrolling_k():
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        driver_k.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [87]:
driver_k = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_k, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_k.get(route_link)
    time.sleep(2)
    
    route_links = driver_k.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_k()

    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_k.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_k.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_k.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [88]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

103 103 103 103 103 103 103 103 103 103


In [89]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_kerala = pd.DataFrame(dict)
df_kerala           #526

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1029,Super Fast Non AC Seater (2+3),00:00,05h 35m,05:35,3.0,INR 269,38 Seats available
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 2518,Swift Deluxe Non AC Air Bus (2+2),00:11,04h 43m,04:54,1.2,INR 401,2 Seats available
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 2368,Low Floor AC Seater 2+2,00:30,05h 19m,05:49,3.9,INR 383,7 Seats available
3,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 698,Super Fast Non AC Seater (2+3),00:45,04h 54m,05:39,1.4,INR 230,15 Seats available
4,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 2820,Super Fast Non AC Seater (2+3),00:55,04h 55m,05:50,3.0,INR 269,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
98,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Murahara Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),00:29,05h 16m,05:45,2.6,1140,7 Seats available
99,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Madhavi Travels,A/C Semi Sleeper (2+2),00:45,06h 00m,06:45,2.8,INR 999,12 Seats available
100,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Ubc travels,A/C Seater/Sleeper (2+1),01:45,03h 55m,05:40,2.6,INR 1200,48 Seats available
101,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Galaxy Tours,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),22:30,03h 30m,02:00,2.6,INR 2099,49 Seats available


In [90]:
df_kerala[df_kerala['Route_name']=='Bangalore to Kozhikode']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [91]:
# df_kerala.to_csv('Kerala.csv')

# 3. Goa - KTCL

In [29]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [30]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        ktcl_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in ktcl_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [32]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/pune-to-goa, Name: Pune to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-pune, Name: Goa to Pune
Link: https://www.redbus.in/bus-tickets/mumbai-to-goa, Name: Mumbai to Goa
Link: https://www.redbus.in/bus-tickets/bangalore-to-goa, Name: Bangalore to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-bangalore, Name: Goa to Bangalore
Link: https://www.redbus.in/bus-tickets/goa-to-mumbai, Name: Goa to Mumbai
Link: https://www.redbus.in/bus-tickets/pandharpur-to-goa, Name: Pandharpur to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-pandharpur, Name: Goa to Pandharpur
Link: https://www.redbus.in/bus-tickets/solapur-to-goa, Name: Solapur to Goa
Link: https://www.redbus.in/bus-tickets/calangute-goa-to-goa-airport, Name: Calangute (goa) to Goa Airport
Link: https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra, Name: Goa to Kolhapur(Maharashtra)
Link: https://www.redbus.in/bus-tickets/goa-to-solapur, Name: Goa to Solapur
Link: https://w

In [33]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("goa_route_details.csv")
df.shape

(34, 2)

In [34]:
def scrolling_kt():
    scrolling = True
    while scrolling:
        old_page_source = driver_kt.page_source
        driver_kt.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_kt.page_source
        
        if new_page_source == old_page_source:
            scrolling = False
        else:
            old_page_source = new_page_source

In [35]:
driver_kt = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_kt, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_kt.get(route_link)
    time.sleep(2)
    
    route_links = driver_kt.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_kt()

    bus_name = driver_kt.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_kt.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_kt.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_kt.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_kt.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_kt.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_kt.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_kt.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with 

In [36]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

437 437 437 437 437 437 437 437 437 437


In [37]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_goa = pd.DataFrame(dict)
df_goa                                             #531

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:00,12h 10m,07:10,3.8,INR 900,18 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:30,10h 00m,07:30,4.6,INR 599,15 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),23:00,09h 45m,08:45,4.5,INR 555,22 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdIntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:00,11h 05m,08:05,4.3,INR 399,30 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:35,10h 55m,08:30,4.3,INR 701,10 Seats available
...,...,...,...,...,...,...,...,...,...,...
432,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),16:00,00h 25m,16:25,3.7,INR 150,47 Seats available
433,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),17:30,00h 30m,18:00,3.7,INR 150,47 Seats available
434,Goa to Shivamogga,https://www.redbus.in/bus-tickets/goa-to-shimoga,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,15:00,10h 03m,01:03,3.7,INR 450,38 Seats available
435,Goa to Shivamogga,https://www.redbus.in/bus-tickets/goa-to-shimoga,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,09h 00m,02:30,3.7,INR 700,13 Seats available


In [38]:
df_goa[df_goa['Route_name']=='Pune to Goa']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:00,12h 10m,07:10,3.8,INR 900,18 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:30,10h 00m,07:30,4.6,INR 599,15 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),23:00,09h 45m,08:45,4.5,INR 555,22 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdIntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:00,11h 05m,08:05,4.3,INR 399,30 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:35,10h 55m,08:30,4.3,INR 701,10 Seats available
5,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Amrutyog,A/C Sleeper (2+1),22:30,11h 00m,09:30,4.3,INR 701,24 Seats available
6,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Gujarat Travels,AC Sleeper (2+1),00:05,10h 06m,10:11,3.3,540,36 Seats available
7,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray National Express,NON A/C Sleeper (2+1),20:35,10h 10m,06:45,4.4,INR 600,20 Seats available
8,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Indumati Travels,A/C Sleeper (2+1),22:35,09h 55m,08:30,4.2,INR 649,26 Seats available
9,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Dolphin Travel House (37),Bharat Benz A/C Sleeper (2+1),20:15,10h 15m,06:30,4.3,INR 710,11 Seats available


In [39]:
# df_goa.to_csv('Goa.csv')

# 4. West Bengal - WBTC (CTC)

In [92]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [93]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        wbtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in wbtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [95]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal, Name: Digha to Barasat (West Bengal)
Link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata, Name: Durgapur to Calcutta
Link: https://www.redbus.in/bus-tickets/digha-to-kolkata, Name: Digha to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-digha, Name: Kolkata to Digha
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha, Name: Barasat (West Bengal) to Digha
Link: https://www.redbus.in/bus-tickets/kolkata-to-suri, Name: Kolkata to Suri
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore, Name: Barasat (West Bengal) to Midnapore
Link: https://www.redbus.in/bus-tickets/midnapore-to-kolkata, Name: Midnapore to Kolkata
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-kolaghat, Name: Barasat (West Bengal) to Kolaghat
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-contai-kanthi, Name: Barasat (West Bengal) to Contai (Kanthi)
Link: https://

In [96]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("westbengal_route_details.csv")
df.shape

(38, 2)

In [97]:
def scrolling_w():
    scrolling = True
    while scrolling:
        old_page_source = driver_w.page_source
        driver_w.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_w.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [98]:
driver_w = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_w, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_w.get(route_link)
    time.sleep(2)
    
    route_links = driver_w.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_w()

    bus_name = driver_w.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_w.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_w.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_w.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_w.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_w.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_w.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_w.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with 

In [414]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

885 885 885 885 885 885 885 885 885 885


In [415]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_westbengal = pd.DataFrame(dict)
df_westbengal                    #885

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,05h 50m,09:50,1.7,INR 165,32 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,04h 30m,10:10,3.4,INR 161,20 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,03h 25m,12:25,3.3,INR 161,5 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,04h 45m,14:45,3.4,INR 161,50 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,"WBTC (CTC) BARASAT-DIGHA via Esplanade, Karuna...",Non AC Seater (2+3),10:15,05h 20m,15:35,3.0,INR 165,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
880,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:55,Non AC Seater (2+3),06:55,03h 50m,10:45,4.0,INR 130,47 Seats available
881,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,03h 55m,16:40,4.0,INR 130,51 Seats available
882,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|14:45,Non AC Seater (2+3),14:45,02h 55m,17:40,4.0,INR 111,50 Seats available
883,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|15:40,Non AC Seater (2+3),15:40,03h 00m,18:40,3.8,INR 111,45 Seats available


In [418]:
df_westbengal[df_westbengal['Route_name']=='Barasat (West Bengal) to Kolaghat']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
586,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|03:45,Non AC Seater (2+3),03:45,01h 40m,05:25,5.0,INR 69,38 Seats available
587,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,01h 35m,06:05,4.3,INR 69,41 Seats available
588,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:00,Non AC Seater (2+3),05:00,01h 35m,06:35,4.0,INR 69,29 Seats available
589,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:35,Non AC Seater (2+3),05:35,01h 35m,07:10,2.5,INR 69,36 Seats available
590,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:00,Non AC Seater (2+3),06:00,01h 35m,07:35,3.3,INR 69,41 Seats available
591,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) Habra-Midnapur - 194|06:25,Non AC Seater (2+3),06:25,01h 40m,08:05,3.6,INR 69,37 Seats available
592,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) Habra-Midnapur - 194|07:00,Non AC Seater (2+3),07:00,01h 45m,08:45,3.3,INR 69,21 Seats available
593,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) Habra- Barasat- Haldia - 180|07:10,Non AC Seater (2+3),07:10,01h 20m,08:30,4.8,INR 69,40 Seats available
594,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) MADHYAMGRAM - DIGHA via Belghoria -...,Non AC Seater (2+3),07:15,01h 30m,08:45,3.6,INR 69,45 Seats available
595,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|08:15,Non AC Seater (2+3),08:15,01h 30m,09:45,1.8,INR 69,47 Seats available


In [25]:
# df_westbengal.to_csv('West_bengal.csv')

# 5. SouthBengal - SBSTC

In [67]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [68]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        ctu_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in ctu_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [70]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata, Name: Durgapur to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-burdwan, Name: Kolkata to Burdwan
Link: https://www.redbus.in/bus-tickets/haldia-to-kolkata, Name: Haldia to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-haldia, Name: Kolkata to Haldia
Link: https://www.redbus.in/bus-tickets/kolkata-to-durgapur, Name: Kolkata to Durgapur (West Bengal)
Link: https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal, Name: Kolkata to Arambagh (West Bengal)
Link: https://www.redbus.in/bus-tickets/midnapore-to-kolkata, Name: Midnapore to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-digha, Name: Kolkata to Digha
Link: https://www.redbus.in/bus-tickets/digha-to-kolkata, Name: Digha to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-bankura, Name: Kolkata to Bankura
Link: https://www.redbus.in/bus-tickets/kolkata-to-midnapore, Name: Kolkata to Midnapore
Link: https://www.re

In [71]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("southbengal_route_details.csv")
df.shape

(41, 2)

In [72]:
def scrolling_s():
    scrolling = True
    while scrolling:
        old_page_source = driver_s.page_source
        driver_s.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_s.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [73]:
driver_s = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_s, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_s.get(route_link)
    time.sleep(2)
    
    route_links = driver_s.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_s()

    bus_name = driver_s.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_s.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_s.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_s.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_s.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_s.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_s.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_s.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [74]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

1343 1343 1343 1343 1343 1343 1343 1343 1343 1343


In [75]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_southbengal = pd.DataFrame(dict)
df_southbengal

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Pammi Travels,Non A/C Seater / Sleeper (2+1),02:00,04h 00m,06:00,3.7,INR 650,17 Seats available
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),02:00,04h 00m,06:00,3.9,463,14 Seats available
2,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),06:10,03h 10m,09:20,4.6,INR 486,4 Seats available
3,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Greenline,Volvo A/C Seater (2+2),06:30,03h 15m,09:45,4.2,INR 486,20 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:40,03h 20m,10:00,4.4,INR 486,18 Seats available
...,...,...,...,...,...,...,...,...,...,...
1338,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 15:30 (MID DEPOT) -...,Non AC Seater (2+3),15:30,08h 30m,00:00,3.9,INR 82,52 Seats available
1339,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-ASANSOL - BELDA - VIA - KOLKATA - 08:50 ...,Non AC Seater (2+3),16:00,08h 00m,00:00,3.9,INR 82,52 Seats available
1340,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - JHARGRAM - VIA - LODHASULI - 1...,Non AC Seater (2+3),16:20,07h 40m,00:00,4.1,INR 82,52 Seats available
1341,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) -...,Non AC Seater (2+3),18:00,06h 00m,00:00,4.3,INR 82,51 Seats available


In [76]:
df_southbengal[df_southbengal['Route_name']=='Kolkata to Nandakumar (west bengal)']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
986,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-GARIA - DIGHA - 04:00 (DIGHA DEPOT) - 911,Non AC Seater (2+3),04:00,20h 00m,00:00,4.5,INR 97,26 Seats available
987,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-GARIA - DIGHA - 04:30 (DIGHA DEPOT) - 1216,Non AC Seater (2+3),04:30,19h 30m,00:00,4.2,INR 97,49 Seats available
988,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-GARIA - DIGHA - 05:00 (DIGHA DEPOT) - 910,Non AC Seater (2+3),05:00,19h 00m,00:00,4.2,INR 97,25 Seats available
989,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-BELGHARIA - DIGHA - 05:15 (BEL DEPOT) - ...,Non AC Seater (2+3),05:15,03h 10m,08:25,4.9,INR 88,39 Seats available
990,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - DIGHA - 06:00 (HWH DEPOT) - 1001,Non AC Seater (2+3),06:00,18h 00m,00:00,4.2,INR 85,48 Seats available
991,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-BADAMTALA - DIGHA - 05:30 - 905,Non AC Seater (2+3),06:00,03h 10m,09:10,4.2,INR 88,34 Seats available
992,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KARUNAMOYEE - DIGHA - VIA DUNLOP - 06:15...,Non AC Seater (2+3),06:15,17h 45m,00:00,4.6,INR 100,6 Seats available
993,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-BELGHARIA - DIGHA - 06:30 (BEL DEPOT) - 900,Non AC Seater (2+3),06:30,03h 10m,09:40,4.9,INR 88,45 Seats available
994,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - HALDIA - 06:30 (BEL DEPOT) - 557,Non AC Seater (2+3),06:30,17h 30m,00:00,4.2,INR 85,31 Seats available
995,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - DIGHA - 06:30 (HWH DEPOT) - 613,Non AC Seater (2+3),06:30,17h 30m,00:00,4.2,INR 85,49 Seats available


In [77]:
# df_southbengal.to_csv('South_Bengal.csv')

# 6. Rajasthan - RSRTC

In [55]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [56]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        pepsu_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in pepsu_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [60]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/udaipur-to-jodhpur, Name: Udaipur to Jodhpur
Link: https://www.redbus.in/bus-tickets/jodhpur-to-ajmer, Name: Jodhpur to Ajmer
Link: https://www.redbus.in/bus-tickets/beawer-to-jaipur, Name: Beawar (Rajasthan) to Jaipur (Rajasthan)
Link: https://www.redbus.in/bus-tickets/sikar-to-jaipur, Name: Sikar to Jaipur (Rajasthan)
Link: https://www.redbus.in/bus-tickets/jaipur-to-jodhpur, Name: Jaipur (Rajasthan) to Jodhpur
Link: https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur, Name: Aligarh (uttar pradesh) to Jaipur (Rajasthan)
Link: https://www.redbus.in/bus-tickets/jaipur-to-aligarh-uttar-pradesh, Name: Jaipur (Rajasthan) to Aligarh (uttar pradesh)
Link: https://www.redbus.in/bus-tickets/jodhpur-to-beawer, Name: Jodhpur to Beawar (Rajasthan)
Link: https://www.redbus.in/bus-tickets/jaipur-to-pilani, Name: Jaipur (Rajasthan) to Pilani
Link: https://www.redbus.in/bus-tickets/kishangarh-to-jaipur, Name: Kishangarh to Jaipur (Rajasthan)
Link

In [61]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("rajasthan_route_details.csv")
df.shape

(18, 2)

In [62]:
def scrolling_r():
    scrolling = True
    while scrolling:
        old_page_source = driver_r.page_source
        driver_r.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_page_source = driver_r.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [63]:
driver_r = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_r, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_r.get(route_link)
    time.sleep(2)
    
    route_links = driver_r.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_r()

    bus_name = driver_r.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_r.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_r.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_r.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_r.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_r.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_r.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_r.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Successfully scraped using Selenium


In [64]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

1635 1635 1635 1635 1635 1635 1635 1635 1635 1635


In [65]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_rajasthan = pd.DataFrame(dict)
df_rajasthan

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174534,Express Non AC Seater 2+3,05:45,06h 15m,12:00,4.2,INR 299,42 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 158798,Super Luxury Volvo AC Seater Pushback 2+2,06:01,04h 59m,11:00,4.3,INR 602,10 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 146395,Express Non AC Seater 2+3,06:45,07h 15m,14:00,4.2,INR 299,42 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174533,Express Non AC Seater 2+3,08:45,07h 15m,16:00,4.2,INR 299,42 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 188347,Super Luxury Volvo AC Seater Pushback 2+2,11:30,05h 00m,16:30,4.2,INR 602,26 Seats available
...,...,...,...,...,...,...,...,...,...,...
1630,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Shree Hare Rama Travels,A/C Seater/Sleeper (2+1),06:00,04h 30m,10:30,2.6,2279,29 Seats available
1631,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,05h 30m,04:30,2.5,INR 300,40 Seats available
1632,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),18:30,04h 50m,23:20,1.8,INR 650,30 Seats available
1633,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,A One Travel Agency,Non A/C Seater / Sleeper (2+1),21:00,05h 30m,02:30,1.6,INR 350,39 Seats available


In [66]:
# df_rajasthan.to_csv('Rajasthan.csv')

# 7. TELANGANA  - TSRTC

In [45]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [46]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        nbtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in nbtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [48]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada, Name: Hyderabad to Vijayawada
Link: https://www.redbus.in/bus-tickets/khammam-to-hyderabad, Name: Khammam to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-khammam, Name: Hyderabad to Khammam
Link: https://www.redbus.in/bus-tickets/hyderabad-to-srisailam, Name: Hyderabad to Srisailam
Link: https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad, Name: Karimnagar to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar, Name: Hyderabad to Karimnagar
Link: https://www.redbus.in/bus-tickets/hyderabad-to-mancherial, Name: Hyderabad to Mancherial
Link: https://www.redbus.in/bus-tickets/hyderabad-to-nirmal, Name: Hyderabad to Nirmal
Link: https://www.redbus.in/bus-tickets/hyderabad-to-adilabad, Name: Hyderabad to Adilabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-ongole, Name: Hyderabad to Ongole
Link: https://www.redbus.in/bus-tickets/kothagudem-to-hyderabad, Name: Kothagudem to

In [49]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("telangana_route_details.csv")
df.shape

(26, 2)

In [50]:
def scrolling_t():
    scrolling = True
    while scrolling:
        old_page_source = driver_t.page_source
        driver_t.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_t.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [51]:
driver_t = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_t, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_t.get(route_link)
    time.sleep(2)
    
    route_links = driver_t.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(3)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException,ElementClickInterceptedException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_t()

    bus_name = driver_t.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_t.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_t.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_t.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_t.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_t.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_t.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_t.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Successfully scraped using Selenium


In [52]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

1770 1770 1770 1770 1770 1770 1770 1770 1770 1770


In [53]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_telangana = pd.DataFrame(dict)
df_telangana

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 351880,VENNELA (A.C. SLEEPER),00:40,05h 25m,06:05,4.7,INR 737,21 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,06h 15m,07:15,3.8,INR 436,33 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,05h 24m,06:49,3.9,INR 781,27 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:45,06h 15m,08:00,4.5,INR 436,33 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:20,05h 20m,07:40,2.9,INR 781,12 Seats available
...,...,...,...,...,...,...,...,...,...,...
1765,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,KKaveri Travels,A/C Sleeper (2+1),21:00,07h 30m,04:30,2.8,INR 890,27 Seats available
1766,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Sahasra Tours and Travels,Non A/C Seater / Sleeper (2+1),21:00,07h 30m,04:30,2.1,INR 449,21 Seats available
1767,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sai Durga Travels,Non A/C Seater / Sleeper (2+1),23:45,07h 15m,07:00,1.6,INR 5000,1 Seat available
1768,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Cherry tours&travels,Non A/C Seater / Sleeper (2+1),21:45,07h 15m,05:00,,INR 640,40 Seats available


In [54]:
# df_telangana.to_csv('Telangana.csv')

# 8. Uttar Pradesh - UPSRTC

In [100]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [101]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        bsrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in bsrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [103]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/delhi-to-bareilly, Name: Delhi to Bareilly
Link: https://www.redbus.in/bus-tickets/bareilly-to-delhi, Name: Bareilly to Delhi
Link: https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi, Name: Aligarh (uttar pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/delhi-to-aligarh-uttar-pradesh, Name: Delhi to Aligarh (uttar pradesh)
Link: https://www.redbus.in/bus-tickets/lucknow-to-allahabad, Name: Lucknow to Allahabad
Link: https://www.redbus.in/bus-tickets/lucknow-to-delhi, Name: Lucknow to Delhi
Link: https://www.redbus.in/bus-tickets/delhi-to-farrukhabad-up, Name: Delhi to Farrukhabad (Uttar Pradesh)
Link: https://www.redbus.in/bus-tickets/farrukhabad-up-to-delhi, Name: Farrukhabad (Uttar Pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/badaun-to-delhi, Name: Badaun to Delhi
Link: https://www.redbus.in/bus-tickets/allahabad-to-lucknow, Name: Allahabad to Lucknow
Link: https://www.redbus.in/bus-tickets/lucknow-to-agra, N

In [104]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("uttarpradesh_route_details.csv")
df.shape

(49, 2)

In [105]:
def scrolling_u():
    scrolling = True
    while scrolling:
        old_page_source = driver_u.page_source
        driver_u.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_u.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [106]:
driver_u = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_u, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_u.get(route_link)
    time.sleep(2)
    
    route_links = driver_u.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException,NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_u()

    bus_name = driver_u.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_u.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_u.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_u.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_u.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_u.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_u.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_u.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [107]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

2538 2538 2538 2538 2538 2538 2538 2538 2538 2538


In [108]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_uttarpradesh = pd.DataFrame(dict)
df_uttarpradesh

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,R S YADAV SMART BUS PRIVATE LIMITED,VE A/C Sleeper (2+1),22:15,06h 00m,04:15,4.4,759,25 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,New Khaira TPT,AC Sleeper (2+1),20:30,06h 20m,02:50,3.3,INR 900,29 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,New Khaira Transport,A/C Seater / Sleeper (2+2),21:00,06h 30m,03:30,1.8,INR 450,33 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,Rahul Travels,AshokLeyland Stile A/C,21:00,05h 00m,02:00,1.9,INR 699,24 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,SHRI RAM TRAVELS,A/C Seater (2+2),23:59,05h 16m,05:15,3.9,INR 740,18 Seats available
...,...,...,...,...,...,...,...,...,...,...
2533,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - ETH0038,Ordinary Non AC Seater 2+3,09:55,06h 00m,15:55,3.3,INR 345,52 Seats available
2534,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - BRT0309,Ordinary Non AC Seater 2+3,10:05,06h 54m,16:59,,INR 371,52 Seats available
2535,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - SBD0108,Janrath AC Seater 2+2,10:30,05h 30m,16:00,4.2,INR 501,36 Seats available
2536,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - GRH0060,Ordinary Non AC Seater 2+3,10:31,05h 29m,16:00,3.3,INR 371,52 Seats available


In [649]:
# df_uttarpradesh.to_csv('UttarPradesh.csv')

# 9. Himachal Pradesh - HRTC

In [109]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [110]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        kaac_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in kaac_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [112]:
kaac = "div.route_details a.route"
Route_Names,Route_links = route_link_name(kaac)

Link: https://www.redbus.in/bus-tickets/delhi-to-shimla, Name: Delhi to Shimla
Link: https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh, Name: Chandigarh to Hamirpur (Himachal Pradesh)
Link: https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh, Name: Hamirpur (Himachal Pradesh) to Chandigarh
Link: https://www.redbus.in/bus-tickets/shimla-to-delhi, Name: Shimla to Delhi
Link: https://www.redbus.in/bus-tickets/delhi-to-chandigarh, Name: Delhi to Chandigarh
Link: https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-delhi, Name: Hamirpur (Himachal Pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/chamba-himachal-pradesh-to-chandigarh, Name: Chamba (Himachal Pradesh) to Chandigarh
Link: https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh, Name: Delhi to Hamirpur (Himachal Pradesh)
Link: https://www.redbus.in/bus-tickets/chandigarh-to-dharamshala, Name: Chandigarh to Dharamshala (Himachal Pradesh)
Link: https://ww

In [113]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("himachalpradesh_route_details.csv")
df.shape

(35, 2)

In [114]:
def scrolling_hp():
    scrolling = True
    while scrolling:
        old_page_source = driver_hp.page_source
        driver_hp.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_page_source = driver_hp.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [115]:
driver_hp = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_hp, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_hp.get(route_link)
    time.sleep(2)
    
    route_links = driver_hp.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException):
        print("Button not found, continuing with scraping...")
    
    time.sleep(4)
    
    scrolling_hp()
    
    bus_name = driver_hp.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_hp.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_hp.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_hp.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_hp.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_hp.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_hp.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_hp.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [116]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

1033 1033 1033 1033 1033 1033 1033 1033 1033 1033


In [117]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_himachalpradesh = pd.DataFrame(dict)
df_himachalpradesh

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 657,Ordinary 3+2 Non AC Seater,11:00,11h 00m,22:00,2.7,INR 512,33 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 515,Ordinary 3+2 Non AC Seater,12:30,10h 20m,22:50,4.2,INR 512,36 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 551,A/C Executive (2+3),15:30,10h 00m,01:30,3.5,INR 632,42 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,10h 10m,03:00,4.2,INR 512,27 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,10h 16m,04:15,2.5,INR 512,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
1028,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Heera Himachal Holidays,Volvo A/C Semi Sleeper (2+2),17:30,14h 30m,08:00,3.0,INR 1399,29 Seats available
1029,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,SIDDHI VINAYAK TRAVELS,Volvo A/C Semi Sleeper (2+2),18:00,13h 00m,07:00,2.9,749,23 Seats available
1030,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Yatrabybus.com,Bharat Benz A/C Semi Sleeper (2+2),18:00,10h 00m,04:00,2.2,INR 1199,49 Seats available
1031,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Royal Travels Hills,Scania Multi-Axle AC Semi Sleeper (2+2),21:00,10h 30m,07:30,1.9,INR 1099,49 Seats available


In [118]:
# df_himachalpradesh.to_csv('HimachalPradesh.csv')

# 10. Jammu - JKRTC

In [119]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/jksrtc");

time.sleep(5)

driver.maximize_window()

In [120]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        jkrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in jkrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [123]:
page = "div.route_details a.route"
Route_Names,Route_links = route_link_name(page)

Link: https://www.redbus.in/bus-tickets/srinagar-to-jammu, Name: Srinagar to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/delhi-to-srinagar, Name: Delhi to Srinagar
Link: https://www.redbus.in/bus-tickets/jammu-to-srinagar, Name: Jammu (j and k) to Srinagar
Link: https://www.redbus.in/bus-tickets/katra-to-jammu, Name: Katra (jammu and kashmir) to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/jammu-to-poonch, Name: Jammu (j and k) to Poonch
Link: https://www.redbus.in/bus-tickets/mendhar-j-k-to-jammu, Name: Mendhar (J & K) to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/jammu-to-chandigarh, Name: Jammu (j and k) to Chandigarh
Link: https://www.redbus.in/bus-tickets/chandigarh-to-jammu, Name: Chandigarh to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/kishtwar-to-jammu, Name: Kishtwar to Jammu (j and k)
No more pages to scroll


In [122]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("jammu_route_details.csv")
df.shape

(9, 2)

In [124]:
def scrolling_j():
    scrolling = True
    while scrolling:
        old_page_source = driver_j.page_source
        driver_j.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_page_source = driver_j.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [125]:
driver_j = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_j, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_j.get(route_link)
    time.sleep(2)
    
    route_links = driver_j.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
    
    time.sleep(4)

    scrolling_j()

    bus_name = driver_j.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_j.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_j.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_j.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_j.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_j.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_j.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_j.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [126]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

24 24 24 24 24 24 24 24 24 24


In [141]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_jammu = pd.DataFrame(dict)
df_jammu                  #105

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...,Jamna Travels-Jammu,Non AC Seater (2+2),19:00,11h 45m,06:45,2.2,600,31 Seats available
1,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...,New Pal Travels,NON A/C Seater / Sleeper (2+2),19:00,10h 00m,05:00,1.8,719,41 Seats available
2,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,2.0,719,36 Seats available
3,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,Jamna Travels-Jammu,Non AC Seater (2+2),21:30,10h 30m,08:00,1.4,508,31 Seats available
4,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),16:30,10h 00m,02:30,1.0,INR 699,40 Seats available
...,...,...,...,...,...,...,...,...,...,...
172,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),05:40,07h 50m,13:30,1.4,INR 400,27 Seats available
173,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),07:00,08h 00m,15:00,1.4,INR 400,27 Seats available
174,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),08:50,08h 40m,17:30,1.4,INR 400,27 Seats available
175,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),09:40,07h 50m,17:30,1.4,INR 400,27 Seats available


In [142]:
df_jammu[df_jammu['Route_name']=='Jammu (j and k) to Chandigarh']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
26,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Chandigarh Transport Undertaking (CTU) - 165941,HVAC Seater (2+3),08:50,09h 35m,18:25,3.3,INR 638.8,46 Seats available
27,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Sleeper (2+1),21:15,07h 15m,04:30,4.7,693,34 Seats available
28,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),20:30,07h 15m,03:45,4.7,521,36 Seats available
29,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,AdHarikesh Tour N Travels,A/C Sleeper (2+1),21:10,07h 20m,04:30,4.2,481,19 Seats available
30,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),20:50,07h 30m,04:20,4.4,755,40 Seats available
...,...,...,...,...,...,...,...,...,...,...
92,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Travel Point,A/C Seater / Sleeper (2+2),18:30,07h 45m,02:15,3.4,904,50 Seats available
93,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Khurana Bus Service,Volvo A/C (2+2),13:30,08h 40m,22:10,3.3,1020,36 Seats available
94,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Khurana Bus Service,Volvo A/C (2+2),13:30,08h 20m,21:50,3.3,1020,36 Seats available
95,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Apex Bus Travels,A/C Seater / Sleeper (2+2),20:00,05h 15m,01:15,2.7,1799,40 Seats available


In [143]:
# df_jammu.to_csv('Jammu.csv') 

## Combining all the State DataFrame to one --> insert into Database --> Clean the DATA

In [672]:
# Red_bus_df = pd.concat([df_andhra,df_kerala,df_goa,df_westbengal,df_southbengal,df_rajasthan,df_telangana,
#                       df_uttarpradesh,df_himachalpradesh,df_jammu],ignore_index=True)

In [3]:
#csv reading
Red_bus_df = pd.read_csv("C:/Users/nagar/Documents/Guvi Projects/Red_bus_project_01_files/Red_bus_df.csv")

In [4]:
Red_bus_df

,Unnamed: 0,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.6,410.0,42
1,1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,450.0,30
2,2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450.0,33
3,3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),23:25,05h 45m,05:10,4.4,383.9,45
4,4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),22:35,06h 40m,05:15,4.7,332.0,29
...,...,...,...,...,...,...,...,...,...,...,...
12991,12991,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),21:30,06h 00m,03:30,2.1,539.0,39
12992,12992,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),20:20,07h 10m,03:30,2.4,595.0,38
12993,12993,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),22:25,07h 10m,05:35,2.4,595.0,39
12994,12994,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,A/C Seater/Sleeper (2+1),23:50,07h 40m,07:30,2.4,619.0,30


In [69]:
Red_bus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12996 entries, 0 to 12995
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      12996 non-null  int64  
 1   Route_name      12996 non-null  object 
 2   Route_link      12996 non-null  object 
 3   Bus_name        12996 non-null  object 
 4   Bus_type        12996 non-null  object 
 5   Depature_time   12996 non-null  object 
 6   Duration        12996 non-null  object 
 7   Reaching_time   12996 non-null  object 
 8   Rating          12996 non-null  float64
 9   Price           12996 non-null  float64
 10  Seat_Available  12996 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1.1+ MB


In [71]:
# Red_bus_df[Red_bus_df['Rating']=='New']

In [689]:
Red_bus_df['Rating'] = pd.to_numeric(Red_bus_df['Rating'],errors='coerce')
rating_mean = round(Red_bus_df['Rating'].mean(),1)
Red_bus_df['Rating'].fillna(rating_mean,inplace=True)
Red_bus_df['Rating'] = Red_bus_df['Rating'].astype(float)

C:\Users\nagar\AppData\Local\Temp\ipykernel_9040\1755810501.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Red_bus_df['Rating'].fillna(rating_mean,inplace=True)


In [691]:
# Red_bus_df.dtypes

In [696]:
Red_bus_df['Price'] = Red_bus_df['Price'].str.replace('INR','')
Red_bus_df['Price'] = Red_bus_df['Price'].astype(float)

In [697]:
Red_bus_df.dtypes

Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available     object
dtype: object

In [703]:
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].str.replace(r' Seats? available', '', regex=True)
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].astype(int)

In [72]:
Red_bus_df.dtypes

Unnamed: 0          int64
Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available      int64
dtype: object

In [11]:
Red_bus_df = Red_bus_df.drop(columns=['Unnamed: 0'])

In [75]:
# Red_bus_df.to_csv('Red_bus_df.csv')

# Connect to Database

In [6]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='@neSre11',database='redbus')

In [7]:
myconnection.cursor()

In [16]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS bus_routes (
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, 
        route_name TEXT, 
        route_link TEXT, 
        bus_name TEXT, 
        bus_type TEXT,
        departing_time varchar(30), 
        duration TEXT, 
        reaching_time varchar(30),
        star_rating FLOAT, 
        price DECIMAL(10, 2), 
        seats_available INT
    );
    """

    # Execute the query
    cursor.execute(create_table_query)
    myconnection.commit()

In [8]:
query = """
    INSERT INTO redbus.bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

In [12]:
Red_bus_df.dtypes

Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available      int64
dtype: object

In [13]:
for i in range(len(Red_bus_df)):
    data = tuple(Red_bus_df.iloc[i])      # Skipping the 'index: 0' column
#     print(data)
    myconnection.cursor().execute(query, data)

In [14]:
myconnection.commit()

In [15]:
pd.read_sql_query('''select * from redbus.bus_routes''',myconnection)

C:\Users\nagar\AppData\Local\Temp\ipykernel_12788\3963494119.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''select * from redbus.bus_routes''',myconnection)


,id,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available
0,1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.6,410.0,42
1,2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,450.0,30
2,3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450.0,33
3,4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),23:25,05h 45m,05:10,4.4,384.0,45
4,5,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),22:35,06h 40m,05:15,4.7,332.0,29
...,...,...,...,...,...,...,...,...,...,...,...
12991,12992,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),21:30,06h 00m,03:30,2.1,539.0,39
12992,12993,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),20:20,07h 10m,03:30,2.4,595.0,38
12993,12994,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),22:25,07h 10m,05:35,2.4,595.0,39
12994,12995,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,A/C Seater/Sleeper (2+1),23:50,07h 40m,07:30,2.4,619.0,30
